In [1]:
from multiprocessing import Pool, cpu_count
import gc; gc.enable()
import pandas as pd
import numpy as np

train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/sample_submission_zero.csv')

transactions = pd.read_csv('cleaned_input/transactions.csv', usecols=['msno'])
transactions = pd.DataFrame(transactions['msno'].value_counts().reset_index())
transactions.columns = ['msno','trans_count']
train = pd.merge(train, transactions, how='left', on='msno')
test = pd.merge(test, transactions, how='left', on='msno')
transactions = []; print('transaction merge...')

user_logs = pd.read_csv('input/user_logs.csv', usecols=['msno'])
user_logs = pd.DataFrame(user_logs['msno'].value_counts().reset_index())
user_logs.columns = ['msno','logs_count']
train = pd.merge(train, user_logs, how='left', on='msno')
test = pd.merge(test, user_logs, how='left', on='msno')
user_logs = []; print('user logs merge...')

members = pd.read_csv('cleaned_input/members.csv')
train = pd.merge(train, members, how='left', on='msno')
test = pd.merge(test, members, how='left', on='msno')
members = []; print('members merge...') 

transaction merge...
user logs merge...
members merge...


In [2]:
transactions = pd.read_csv('input/transactions.csv')
transactions = transactions.sort_values(by=['transaction_date'], ascending=[False]).reset_index(drop=True)
transactions = transactions.drop_duplicates(subset=['msno'], keep='first')

train = pd.merge(train, transactions, how='left', on='msno')
test = pd.merge(test, transactions, how='left', on='msno')
transactions=[]

In [3]:
train.head()

,msno,is_churn,trans_count,logs_count,Unnamed: 0,city,bd,gender,registered_via,registration_init_year,...,expiration_date_month,expiration_date_date,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,2,26.0,4396024.0,18.0,36.0,female,9.0,2005.0,...,9.0,7.0,38,30,149,149,0,20170107,20170206,0
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,23,521.0,1941053.0,10.0,38.0,male,9.0,2005.0,...,3.0,21.0,39,30,149,149,1,20170224,20170321,1
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,10,237.0,1587455.0,11.0,27.0,female,9.0,2005.0,...,2.0,3.0,39,30,149,149,1,20170112,20170203,1
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,2,735.0,1500627.0,13.0,23.0,female,9.0,2005.0,...,9.0,26.0,38,410,1788,1788,0,20160106,20170220,0
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,8,758.0,2600269.0,3.0,27.0,male,9.0,2005.0,...,9.0,27.0,38,30,149,149,0,20170128,20170227,0


In [4]:
def transform_df(df):
    df = pd.DataFrame(df)
    df = df.sort_values(by=['date'], ascending=[False])
    df = df.reset_index(drop=True)
    df = df.drop_duplicates(subset=['msno'], keep='first')
    return df

def transform_df2(df):
    df = df.sort_values(by=['date'], ascending=[False])
    df = df.reset_index(drop=True)
    df = df.drop_duplicates(subset=['msno'], keep='first')
    return df

df_iter = pd.read_csv('input/user_logs.csv', low_memory=False, iterator=True, chunksize=10000000)
last_user_logs = []
i = 0 #~400 Million Records - starting at the end but remove locally if needed
for df in df_iter:
    if i>35:
        if len(df)>0:
            print(df.shape)
            p = Pool(cpu_count())
            df = p.map(transform_df, np.array_split(df, cpu_count()))   
            df = pd.concat(df, axis=0, ignore_index=True).reset_index(drop=True)
            df = transform_df2(df)
            p.close(); p.join()
            last_user_logs.append(df)
            print('...', df.shape)
            df = []
    i+=1

last_user_logs = pd.concat(last_user_logs, axis=0, ignore_index=True).reset_index(drop=True)
last_user_logs = transform_df2(last_user_logs)

train = pd.merge(train, last_user_logs, how='left', on='msno')
test = pd.merge(test, last_user_logs, how='left', on='msno')
last_user_logs=[]

(10000000, 9)
... (1616917, 9)
(10000000, 9)
... (1533539, 9)
(10000000, 9)
... (1353720, 9)
(2106543, 9)
... (429234, 9)


In [5]:
print(train.shape)
print(test.shape)

train.to_csv("cleaned_input/train_merged.csv")
test.to_csv("cleaned_input/test_merged.csv")

(992931, 31)
(970960, 31)


In [6]:
data = pd.concat(( train, test ))
train_cutoff = len(train)




In [7]:
#if missing >3 cells, remove row 
data = data.dropna(thresh=3)
data.isnull().any()

msno                       False
is_churn                   False
trans_count                 True
logs_count                  True
Unnamed: 0                  True
city                        True
bd                          True
gender                      True
registered_via              True
registration_init_year      True
registration_init_month     True
registration_init_date      True
expiration_date_year        True
expiration_date_month       True
expiration_date_date        True
payment_method_id           True
payment_plan_days           True
plan_list_price             True
actual_amount_paid          True
is_auto_renew               True
transaction_date            True
membership_expire_date      True
is_cancel                   True
date                        True
num_25                      True
num_50                      True
num_75                      True
num_985                     True
num_100                     True
num_unq                     True
total_secs

In [8]:
#fill in missing data points and encode variables
from helpers import *

nan_zero = {"logs_count":0}
nan_dif = {"city":-1, "registered_via":-1, "payment_method_id":-1, "is_auto_renew":-1, "is_cancel":-1}
nan_med = {"bd": 'med', "registration_init_date": 'med', "expiration_date_month":'med', "expiration_date_date":'med',
           "expiration_date_year": 'med', "registration_init_year":'med', "registration_init_month":'med', 
           "registration_init_date":'med', "date":'med', "num_25": 'med', "num_50": 'med', "num_75": 'med', "num_985": 'med', 
           "num_100": 'med', "num_unq": 'med', "Unnamed: 0": 'med', "total_secs": 'med', "trans_count":'med', 'transaction_date':'med',
          'membership_expire_date':'med', 'plan_list_price':'med', 'actual_amount_paid':'med', 'payment_plan_days':'med'}
nan_cat = {"gender":"missing"}

data = data.drop(["Unnamed: 0"])
data = to_fill_na(data, nan_zero)
data = to_fill_na(data, nan_dif)
data = to_fill_na(data, nan_med)
data = to_fill_na(data, nan_cat)
train.head()


/Users/James/anaconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,msno,is_churn,trans_count,logs_count,Unnamed: 0,city,bd,gender,registered_via,registration_init_year,...,membership_expire_date,is_cancel,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,2,26.0,4396024.0,18.0,36.0,female,9.0,2005.0,...,20170206,0,20170121.0,4.0,0.0,2.0,5.0,76.0,74.0,19799.702
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,23,521.0,1941053.0,10.0,38.0,male,9.0,2005.0,...,20170321,1,20170215.0,0.0,0.0,0.0,0.0,3.0,3.0,796.916
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,10,237.0,1587455.0,11.0,27.0,female,9.0,2005.0,...,20170203,1,20161225.0,61.0,21.0,9.0,11.0,44.0,130.0,15845.692
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,2,735.0,1500627.0,13.0,23.0,female,9.0,2005.0,...,20170220,0,20170103.0,11.0,3.0,4.0,1.0,18.0,21.0,5085.345
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,8,758.0,2600269.0,3.0,27.0,male,9.0,2005.0,...,20170227,0,20170208.0,0.0,0.0,1.0,1.0,98.0,100.0,25252.857


In [9]:
data.isnull().any()

msno                       False
is_churn                   False
trans_count                False
logs_count                 False
Unnamed: 0                 False
city                       False
bd                         False
gender                     False
registered_via             False
registration_init_year     False
registration_init_month    False
registration_init_date     False
expiration_date_year       False
expiration_date_month      False
expiration_date_date       False
payment_method_id           True
payment_plan_days           True
plan_list_price             True
actual_amount_paid          True
is_auto_renew               True
transaction_date            True
membership_expire_date      True
is_cancel                   True
date                       False
num_25                     False
num_50                     False
num_75                     False
num_985                    False
num_100                    False
num_unq                    False
total_secs

In [ ]:
#feature engineering 

df_transactions['discount'] = df_transactions['plan_list_price'] - df_transactions['actual_amount_paid']
df_transactions['is_discount'] = df_transactions.discount.apply(lambda x: 1 if x > 0 else 0)
df_transactions['amt_per_day'] = df_transactions['actual_amount_paid'] / df_transactions['payment_plan_days']
date_cols = ['transaction_date', 'membership_expire_date']
for col in date_cols:
    df_transactions[col] = pd.to_datetime(df_transactions[col], format='%Y%m%d')
    
#--- difference in days ---
df_transactions['membership_duration'] = df_transactions.membership_expire_date - df_transactions.transaction_date
df_transactions['membership_duration'] = df_transactions['membership_duration'] / np.timedelta64(1, 'D')
df_transactions['membership_duration'] = df_transactions['membership_duration'].astype(int)

 
#---difference in months ---
#df_transactions['membership_duration_M'] = (df_transactions.membership_expire_date - df_transactions.transaction_date)/ np.timedelta64(1, 'M')
#df_transactions['membership_duration_M'] = round(df_transactions['membership_duration_M']).astype(int)
#df_transactions['membership_duration_M'].head()

date_cols = ['registration_init_time', 'expiration_date']

for col in date_cols:
    df_members[col] = pd.to_datetime(df_members[col], format='%Y%m%d')

#--- difference in days ---
df_members['registration_duration'] = df_members.expiration_date - df_members.registration_init_time
df_members['registration_duration'] = df_members['registration_duration'] / np.timedelta64(1, 'D')
df_members['registration_duration'] = df_members['registration_duration'].astype(int)

#---difference in months ---
#df_members['registration_duration_M'] = (df_members.expiration_date - df_members.registration_init_time)/ np.timedelta64(1, 'M')
#df_members['registration_duration_M'] = round(df_members['registration_duration_M']).astype(int)

df_comb['reg_mem_duration'] = df_comb['registration_duration'] - df_comb['membership_duration']
#df_comb['reg_mem_duration_M'] = df_comb['registration_duration_M'] - df_comb['membership_duration_M']
df_comb.head()


df_comb['notAutorenew_&_cancel'] = ((df_comb.is_auto_renew == 0) == (df_comb.is_cancel == 1)).astype(np.int8)
df_comb['notAutorenew_&_cancel'].unique()

df_comb['long_time_user'] = (((df_comb['registration_duration'] / 365).astype(int)) > 1).astype(int)



In [ ]:
train = data[:train_cutoff]
test = data[train_cutoff:]
print(train.shape)
print(test.shape)

train.to_csv("cleaned_input/train_consolidated.csv")
test.to_csv("cleaned_input/test_consolidated.csv")